In [ ]:
import pyspark.sql.functions as F

from pyspark.sql import DataFrame

from src.vars import TABLE_CONFIGS, LANDING_BASE_PATH
from src.commons import merge_into_bronze_changelog


def read_landing_for_table(table_name: str) -> DataFrame:
    path = f"{LANDING_BASE_PATH}/{table_name}"
    df = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("delimiter", ";")
        .csv(path)
    )
    return df


def run_bronze_for_table(table_name: str) -> None:
    if table_name not in TABLE_CONFIGS:
        raise ValueError(f"Tabela '{table_name}' não está configurada.")
    conf = TABLE_CONFIGS[table_name]
    business_key_cols = conf["business_key"]
    df = read_landing_for_table(table_name)
    if df.rdd.isEmpty():
        return
    if "data_ref" in df.columns:
        df = df.withColumn("data_ref", F.col("data_ref").cast("timestamp"))
    merge_into_bronze_changelog(df, table_name, business_key_cols, "from_landing")


for tbl in TABLE_CONFIGS.keys():
    run_bronze_for_table(tbl)
